In [72]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
data = pd.read_json("training_90_balanced.json")
data_test = pd.read_json("test_10.json")

In [11]:
# pip install gensim
# Taggeddocument for training data
taggedDocuments = []
for index, data_point_post in data.iterrows():
    taggedDocument =  TaggedDocument(words=data.at[index, 'posts'], tags=[data.at[index, 'type']], axis=1)
    taggedDocuments.append(taggedDocument)

Total words:  478
[TaggedDocument(words=['adus', 'hurt', 'weak', 'little', 'kid', 'punishment', 'effective', 'criminal', 'sociopath', 'pedophilic', 'desire', 'sociopath', 'care', 'thing', 'hurt', 'hard', 'wire', 'pedophile', 'thing', 'put', 'solitary', 'environment', 'introspection', 'occur', 'may', 'fix', 'think', 'countryside', 'perfect', 'place', 'rehabilitate', 'case', 'contact', 'one', 'live', 'slow', 'move', 'environment', 'may', 'help', 'build', 'brain', 'farm', 'fish', 'hunt', 'would', 'best', 'method', 'help', 'catch', 'mentally', 'human', 'solitary', 'confinement', 'fix', 'people', 'problem', 'would', 'say', 'best', 'thing', 'treat', 'people', 'kindness', 'remember', 'kid', 'crush', 'old', 'cartoon', 'character', 'kindhearted', 'understand', 'kid', 'develop', 'would', 'imagine', 'pedophile', 'could', 'fall', 'love', 'woman', 'really', 'humble', 'sweet', 'despite', 'appearance', 'child', 'humble', 'sweet', 'heart', 'one', 'ur', 'serious', 'right', 'kind', 'measure', 'joule', '

In [63]:
# Taggeddocument for test data
taggedDocuments_test = []
for index, data_point_post in data_test.iterrows():
    taggedDocument =  TaggedDocument(words=data_test.at[index, 'posts'], tags=[data_test.at[index, 'type']])
    taggedDocuments_test.append(taggedDocument)

In [12]:
# creating the model
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(taggedDocuments)

model.train(taggedDocuments, total_examples=model.corpus_count, epochs=model.epochs)
print("DONE")

DONE


In [13]:
model.save('model.doc2vec')

In [39]:
def vec_for_learning(model, taggedDocuments):
    """
    infering vectors from the model 
    """
    sents = taggedDocuments[0:45865]
    tags, posts = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return tags, posts

In [40]:
# vectorizing the posts and types
type_train, posts_train = vec_for_learning(model, taggedDocuments)

In [64]:
type_test, posts_test = vec_for_learning(model, taggedDocuments_test)

In [53]:
# Applying logistic regression
# C ? = hyperparam

logreg = LogisticRegression(n_jobs=4, C=1e5)
logreg.fit(posts_train, type_train)

LogisticRegression(C=100000.0, n_jobs=4)

In [73]:
# Testing the model
type_pred = logreg.predict(posts_test)
print('Testing accuracy %s' % accuracy_score(type_test, type_pred))
print('Testing F1 score: {}'.format(f1_score(type_test, type_pred, average='weighted')))
print(classification_report(type_test, type_pred))

Testing accuracy 0.7348920524182144
Testing F1 score: 0.7514792425228755
              precision    recall  f1-score   support

           E       0.45      0.70      0.55      2458
           I       0.89      0.74      0.81      8149

    accuracy                           0.73     10607
   macro avg       0.67      0.72      0.68     10607
weighted avg       0.79      0.73      0.75     10607

